In [2]:
import os
import glob
import json
import quickstats
import sys, numpy as np
from quickstats.components import AnalysisBase

%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# make sure hh combination fw path is defined
hh_comb_fw_path = os.environ.get('hh_combination_fw_path', None)

assert hh_comb_fw_path

gen_command = True

In [4]:
from quickstats.maths.numerics import is_integer, pretty_value
from pprint import pprint
def create_job(string, key, suf=''):
    tmp = string.split(',')
    poi1 = tmp[0].split('=')[0]
    poi2 = tmp[1].split('=')[0]
    poi1_tmp = tmp[0].split('=')[1].split('_')
    poi2_tmp = tmp[1].split('=')[1].split('_')
    poi1_range = [float(p) for p in poi1_tmp[0:2]]
    poi1_step = float(poi1_tmp[2])
    poi2_range = [float(p) for p in poi2_tmp[0:2]]
    poi2_step = float(poi2_tmp[2])

    poi1_nsteps = int((poi1_range[1] - poi1_range[0]) / poi1_step)
    poi2_nsteps = int((poi2_range[1] - poi2_range[0]) / poi2_step)

#     # slice poi1 with full poi2 range
#     if poi1_nsteps < poi2_nsteps and poi2_nsteps > 12:
#         poi1, poi1_range, poi1_step, poi1_nsteps, poi2, poi2_range, poi2_step, poi2_nsteps = poi2, poi2_range, poi2_step, poi2_nsteps, poi1, poi1_range, poi1_step, poi1_nsteps
        
    poi_slice_up = np.arange(poi1_range[0], poi1_range[1], poi1_step).round(decimals=2)
    new_dict = {}
    for i in range(poi1_nsteps):
        key_name = key+f'_job{suf}{i+1}'
        value = f"^{poi1}={poi_slice_up[i]}_{pretty_value(poi_slice_up[i]+poi1_step)}_{poi1_step},{poi2}={pretty_value(poi2_range[0])}_{pretty_value(poi2_range[1])}_{poi2_step}^"
        new_dict[key_name] = value
    return new_dict


In [5]:
def return_combine_command(paths, config):
    options = {
        "input_dir"    : paths['output'],
        "resonant_type": config['resonant_type'],
        "channels"     : ",".join(config['channels']),
        "file_expr"    : f"\"{config['file_expr']}\"",
        "config"       : paths['task_options'],
        "scheme"       : paths['correlation_schemes'],
        "parallel"     : config['parallel'],
        "skip-limit"   : ""
    }
    if config["blind"]:
        options["blind"] = ""
    else:
        options["unblind"] = ""
    if config["experimental"]:
        options["experimental"] = ""
    else:
        options["official"] = ""
    if config['cache']:
        options['cache'] = ""
    else:
        options['no-cache'] = ""
    if 'prefix' in config:
        options['prefix'] = config['prefix']
    if config['type'] == 'xsec':
        options['minimizer_options'] = os.path.join(hh_comb_fw_path, "configs/minimizer_options/fix_xs_uncertainty.json")

    command_str = "HHComb combine_ws " + " ".join([f"--{key} {value}" for key, value in options.items()])
    print(command_str+' &')
    return command_str



# Step 1: SM point

In [17]:
# specify the input timestamp to use
timestamp = "20220520_noSgHparam"
withbr = None
type='mu' # 'xsec' 'mu'

paths = {
    'input'              : os.path.join(hh_comb_fw_path, "FullRun2Workspaces", "original", "HHH2022", timestamp),
    'task_options'       : os.path.join(hh_comb_fw_path, "configs", "task_options"       , "HHH2022",
                                        f"nonres_SM.yaml"),
    'correlation_schemes': os.path.join(hh_comb_fw_path, "configs", "correlation_schemes", "HHH2022", 
                                        "nonres_kl_v13.json"),
    'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_20220701_noSgHparam_{type}")
}

config = {
    'resonant_type': 'nonres',
    'channels'     : ['bbbb', 'bbtautau', 'bbyy'],
    'file_expr'    : '<mass[F]>',
    'blind'        : False,
    'cache'        : True,
    'experimental' : True,
    'parallel'     : -1,
    'type'         : type
}
options = {
    "input_dir"    : paths['input'],
    "resonant_type": config['resonant_type'],
    "channels"     : ",".join(config['channels']),
    "outdir"       : paths['output'],
    "file_expr"    : f"\"{config['file_expr']}\"",
    "config"       : paths['task_options'],
    "parallel"     : config['parallel'],
    "skip-limit"   : "",
}

## process channels

In [18]:

if config["blind"]:
    options["blind"] = ""
else:
    options["unblind"] = ""
if config["experimental"]:
    options["experimental"] = ""
else:
    options["official"] : ""
if config['type'] == 'xsec':
    options['minimizer_options'] = os.path.join(hh_comb_fw_path, "configs/minimizer_options/fix_xs_uncertainty.json")
else:
    pass
if config['cache']:
    options['cache'] = ""
else:
    options['no-cache'] = ""

    
    
command_str1 = "HHComb process_channels " + " ".join([f"--{key} {value}" for key, value in options.items()])
print(command_str1)

HHComb process_channels --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/FullRun2Workspaces/original/HHH2022/20220520_noSgHparam --resonant_type nonres --channels bbbb,bbtautau,bbyy --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_mu --file_expr "<mass[F]>" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_SM.yaml --parallel -1 --skip-limit  --unblind  --experimental  --cache 


## combine ws

In [19]:
command_str = return_combine_command(paths, config)

HHComb combine_ws --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_mu --resonant_type nonres --channels bbbb,bbtautau,bbyy --file_expr "<mass[F]>" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_SM.yaml --scheme /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/correlation_schemes/HHH2022/nonres_kl_v13.json --parallel -1 --skip-limit  --unblind  --experimental  --cache  &


## obtain path

In [20]:
rescaled_ws_paths = {}
for channel in config['channels']:
    ws_path = os.path.join(paths['output'], 'rescaled', config['resonant_type'], channel, "0.root")
    rescaled_ws_paths[channel]  = ws_path
# rescaled workspace path for combined workspacee
ws_path = glob.glob(os.path.join(paths['output'], 'combined', config['resonant_type'], "*", "0.root"))[0]
rescaled_ws_paths["combined"] = ws_path
rescaled_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_mu/rescaled/nonres/bbbb/0.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_mu/rescaled/nonres/bbtautau/0.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_mu/rescaled/nonres/bbyy/0.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_mu/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0.root'}

## Step 1.1: Cross section limits

In [21]:
# quickstats cls_limit -i ../../../combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0.root -f *fixmu*=0 --outname fixmu --unblind

## Step 1.2: SM ranking

In [22]:
# from quickstats.components import ExtendedModel
# channel = 'combined'
# model = ExtendedModel(rescaled_ws_paths[channel])
# param_sets = model.get_constrained_nuisance_parameters()


In [23]:
# condor = True
# if condor:
#     print('Save jobrank.txt')
#     original_stdout = sys.stdout
#     sys.stdout = open('jobrank.txt', 'w')
    
# for p in param_sets:
#     if p.GetName().startswith('gamma_'):
#         continue
#     channel_command_str = f'quickstats run_pulls -i {rescaled_ws_paths["combined"]} --poi xsec_br -o {paths["output"]}/ranking/{channel} -p {p.GetName()}'

#     print("Arguments =", channel_command_str.replace(' ', '____'))
#     print("Queue 1")
#     print()

# if condor: 
#     sys.stdout = original_stdout
    

## Step 1.3: SM best fit

In [24]:
expected = False
def run_best_fit(channel):
    minimizer_options = {
        "retry": 2,
        "eps": 1,
    }
    if config['type'] == 'xsec':
        minimizer_options['fix_param'] = "<poi>,THEO_XS_fixmu_*=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0, alpha_THEO_XS_PDFalphas_ggFSMHH*=0, alpha_THEO_XS_SCALEMTop_ggFSMHH*=0, THEO_XS_COMBINED_HH_ggF*=0, THEO_XS_PDFalphas_HH_VBF*=0, THEO_XS_PDFalphas_HH_ggF*=0, THEO_XS_SCALE_HH_VBF*=0,alpha_THEO_XS_PDFalphas_ggf=0,alpha_THEO_XS_PDFalphas_vbf=0,alpha_THEO_XS_SCALEMTop_ggf=0,alpha_THEO_XS_SCALEMTop_vbf=0"
    else:
        minimizer_options['fix_param'] = "<poi>"
    # observed and expected datasets
    datasets = {
        "observed": "combData",
    }
    poi_expr = 'xsec_br'
    print(f"==> Channel: {channel}")
    ws_path = rescaled_ws_paths[channel]
    print(ws_path, poi_expr, datasets["observed"])
    kwargs = {
        "filename" : ws_path,
        "data_name": datasets["observed"],
        "poi_name": poi_expr,
        "config": {**minimizer_options}
    }
    analysis = AnalysisBase(**kwargs)
#     analysis.setup_parameters(fix_param="<poi>")
    analysis.setup_parameters(profile_param=poi_expr)
    
    if expected:
        analysis.generate_standard_asimov(asimov_types=[-2])
        analysis.set_data("asimovData_1_NP_Nominal")
        analysis.load_snapshot("asimovData_1_NP_Nominal")
        
    result = analysis.nll_fit(mode=1, do_minos=False)
    return result

if False:
    results = {}
    for channel in config['channels']+['combined']:
        results[channel] = run_best_fit(channel)
    result_path = os.path.join(paths['output'], 'bestfit', f'result_observed.json')
    os.makedirs(os.path.join(paths['output'], 'bestfit'), exist_ok=True)
    with open(result_path, 'w') as fp:
        json.dump(results, fp, indent=2)

In [25]:
print(results)

NameError: name 'results' is not defined

In [26]:
paths['output']

'/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_mu'

## Step 1.4: SM studies

In [27]:
# # specify the input timestamp to use
# timestamp = "20220520_noSgHparam"
# withbr = None

# for corr in ["nonres_kl_v12.json", "nonres_kl_v12_decor.json", "nonres_kl_v12_decor_corrtop2.json", "nonres_kl_v12_decor_corrtop2exp.json"]:
#     for job in ['mu', 'xsec']:
        
#         paths = {
#             'input'              : os.path.join(hh_comb_fw_path, "FullRun2Workspaces", "original", "HHH2022", timestamp),
#             'task_options'       : os.path.join(hh_comb_fw_path, "configs", "task_options"       , "HHH2022", f"nonres_SM.yaml"),
#             'correlation_schemes': os.path.join(hh_comb_fw_path, "configs", "correlation_schemes", "HHH2022", corr),
#             'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_{timestamp}")
#         }

#         config = {
#             'resonant_type': 'nonres',
#             'channels'     : ['bbbb', 'bbtautau', 'bbyy'],
#             'file_expr'    : '<mass[F]>',
#             'blind'        : False,
#             'cache'        : True,
#             'experimental' : True,
#             'parallel'     : -1,
#             'type'         : job,
#             'prefix'       : corr[:-5]+'_'+job+'_',
#         }
#         options = {
#             "input_dir"    : paths['input'],
#             "resonant_type": config['resonant_type'],
#             "channels"     : ",".join(config['channels']),
#             "outdir"       : paths['output'],
#             "file_expr"    : f"\"{config['file_expr']}\"",
#             "config"       : paths['task_options'],
#             "parallel"     : config['parallel'],
#             "skip-limit"   : "",
#         }

#         if config['type'] == 'xsec':
#             options['minimizer_options'] = os.path.join(hh_comb_fw_path, "configs/minimizer_options/fix_xs_uncertainty.json")



#         if config["blind"]:
#             options["blind"] = ""
#         else:
#             options["unblind"] = ""
#         if config["experimental"]:
#             options["experimental"] = ""
#         else:
#             options["official"] : ""
#         if config['cache']:
#             options['cache'] = ""
#         else:
#             options['no-cache'] = ""


#         command_str1 = "HHComb process_channels " + " ".join([f"--{key} {value}" for key, value in options.items()])
# #         print('\n## process channels', corr, job)
# #         print(command_str1)
#         print('\n## combine ws', corr, job)
#         command_str = return_combine_command(paths, config)
        
        


## 1.5 SM S+B asimov limit

In [30]:
from quickstats.components import AsymptoticCLs

def gen_asimov(channel):
    poi_expr = 'xsec_br'
    ws_path = rescaled_ws_paths[channel]
    output = ws_path.replace('0.root', '0_asimov.root')
    command = f'quickstats generate_standard_asimov --asimov_types 0,1,2 --poi {poi_expr} --data combData -i {ws_path} -o {output} &'
    print(command)
    print()

def fit_asimovData_1_NP_Profile(channel):
    poi_expr = 'xsec_br'
    ws_path = rescaled_ws_paths[channel]
    ws_input = ws_path.replace('0.root', '0_asimov.root')
    limit_output = ws_input.replace('0_asimov.root', f'../../../limits/nonres/{channel}/limits_mu_exp1.json')
#     command = f'quickstats cls_limit -i {ws_input} --poi {poi_expr} --data asimovData_1_NP_Profile --snapshot asimovData_1_NP_Profile --unblind --outname {limit_output}xsec_br &'
#     command = f'quickstats cls_limit -i {ws_input} --poi {poi_expr} --data combData --asimov_data_name asimovData_1_NP_Profile --unblind --outname {limit_output} &'
    command = f'quickstats cls_limit -i {ws_input} --poi {poi_expr} --data combData --unblind --outname {limit_output} --mu_exp 1&'

    print(command)
    print()
    
    
    
for channel in config['channels']+['combined']:
    gen_asimov(channel)
    
print('#'*10)
for channel in config['channels']+['combined']:
    fit_asimovData_1_NP_Profile(channel)

quickstats generate_standard_asimov --asimov_types 0,1,2 --poi xsec_br --data combData -i /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_mu/rescaled/nonres/bbbb/0.root -o /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_mu/rescaled/nonres/bbbb/0_asimov.root &

quickstats generate_standard_asimov --asimov_types 0,1,2 --poi xsec_br --data combData -i /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_mu/rescaled/nonres/bbtautau/0.root -o /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_mu/rescaled/nonres/bbtautau/0_asimov.root &

quickstats generate_standard_asimov --asimov_types 0,1,2 --poi xsec_br --data combData -i /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutori

In [16]:
rescaled_ws_paths['bbyy']

'/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_mu/rescaled/nonres/bbyy/0.root'

## Step 2: Cross section scans

In [83]:
# specify the input timestamp to use
timestamp = "20220520_noSgHparam"
# timestamp = "20220701_with_BR_decorrelation"
# timestamp = "20220701_noSgHparam_xsec/"

paths = {
    'input'              : os.path.join(hh_comb_fw_path, "FullRun2Workspaces", "original", "HHH2022", timestamp),
    'task_options'       : os.path.join(hh_comb_fw_path, "configs", "task_options"       , "HHH2022",
                                        f"nonres_kl_kt_xsection.yaml"),
    'correlation_schemes': os.path.join(hh_comb_fw_path, "configs", "correlation_schemes", "HHH2022", 
                                        "nonres_kl_v13.json"),
    'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_20220701_noSgHparam_xsec")
}

config = {
    'resonant_type': 'nonres',
    'channels'     : ['bbbb', 'bbtautau', 'bbyy'],
    'file_expr'    : '<mass[F]>_kl',
    'blind'        : False,
    'cache'        : True,
    'experimental' : True,
    'parallel'     : -1,
    'type'         : 'xsec'
}
options = {
    "input_dir"    : paths['input'],
    "resonant_type": config['resonant_type'],
    "channels"     : ",".join(config['channels']),
    "outdir"       : paths['output'],
    "file_expr"    : f"\"{config['file_expr']}\"",
    "config"       : paths['task_options'],
    "parallel"     : config['parallel'],
    "skip-limit"   : "",
}

if config["blind"]:
    options["blind"] = ""
else:
    options["unblind"] = ""
if config["experimental"]:
    options["experimental"] = ""
else:
    options["official"] = ""
if config['cache']:
    options['cache'] = ""
else:
    options['no-cache'] = ""
command_str1 = "HHComb process_channels " + " ".join([f"--{key} {value}" for key, value in options.items()])
print(command_str1)

if config['type'] == 'xsec':
    options['fix'] = f"\"THEO_XS_fixmu_*=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0,alpha_THEO_XS_PDFalphas_ggFSMHH*=0,alpha_THEO_XS_SCALEMTop_ggFSMHH*=0,THEO_XS_COMBINED_HH_ggF*=0,THEO_XS_PDFalphas_HH_VBF*=0,THEO_XS_PDFalphas_HH_ggF*=0,THEO_XS_SCALE_HH_VBF*=0\""


HHComb process_channels --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/FullRun2Workspaces/original/HHH2022/20220520_noSgHparam --resonant_type nonres --channels bbbb,bbtautau,bbyy --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_xsec --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_xsection.yaml --parallel -1 --skip-limit  --unblind  --experimental  --cache 


In [84]:
command_str = return_combine_command(paths, config)

HHComb combine_ws --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_xsec --resonant_type nonres --channels bbbb,bbtautau,bbyy --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_xsection.yaml --scheme /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/correlation_schemes/HHH2022/nonres_kl_v13.json --parallel -1 --skip-limit  --unblind  --experimental  --cache  --minimizer_options /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/minimizer_options/fix_xs_uncertainty.json &


In [85]:
os.path.join(paths['output'], 'combined', config['resonant_type'], "*", "0_kl.root")

'/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_xsec/combined/nonres/*/0_kl.root'

In [86]:
rescaled_ws_paths = {}
for channel in config['channels']:
    ws_path = os.path.join(paths['output'], 'rescaled', config['resonant_type'], channel, "0_kl.root")
    rescaled_ws_paths[channel] = ws_path
# rescaled workspace path for combined workspacee
ws_path = glob.glob(os.path.join(paths['output'], 'combined', config['resonant_type'], "*", "0_kl.root"))[0]
rescaled_ws_paths["combined"] = ws_path
rescaled_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_xsec/rescaled/nonres/bbbb/0_kl.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_xsec/rescaled/nonres/bbtautau/0_kl.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_xsec/rescaled/nonres/bbyy/0_kl.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_xsec/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_kl.root'}

In [87]:
# use '_job' + suffix to split scan range
param_expr_maps = {
    'klambda_job1': "^klambda=-10_-5_0.2^",
    'klambda_job2': "^klambda=-5_0_0.2^",
    'klambda_job3': "^klambda=0_5_0.2^",
    'klambda_job4': "^klambda=5_10_0.2^",
    'klambda_job5': "^klambda=10_15_0.2^",
    'klambda_job7': "^klambda=4_7_0.1^",
    'klambda_final': "^klambda=-10_15_0.2,klambda=4_7_0.1^",
    'k2v_job1': "^k2V=-2_0_0.2^",
    'k2v_job1': "^k2V=0_2_0.2^",
    'k2v_job1': "^k2V=2_4_0.2^",
    'k2v_job2': "^k2V=0_1_0.05^",
    'k2v_job3': "^k2V=1_2_0.05^",
    'k2v_final': "^k2V=-2_4_0.2,k2V=0_2_0.05^",
#     'klambdak2v_final': "^klambda=-15_20_0.2,k2V=-2_4_0.2^",
#     'k2vkv_final': "^k2V=-2_6_0.2,kV=-3_3_0.2^",
}

# param_expr_maps.update(create_job('klambda=-15_20_0.2,k2V=-2_4_0.2', 'klambdak2v'))
# param_expr_maps.update(create_job('k2V=-2_6_0.2,kV=-3_-2_0.2', 'k2vkv', suf='a'))
# param_expr_maps.update(create_job('k2V=-2_6_0.2,kV=-2_-1_0.2', 'k2vkv', suf='b'))
# param_expr_maps.update(create_job('k2V=-2_6_0.2,kV=-1_0_0.2', 'k2vkv', suf='c'))
# param_expr_maps.update(create_job('k2V=-2_6_0.2,kV=0_1_0.2', 'k2vkv', suf='d'))
# param_expr_maps.update(create_job('k2V=-2_6_0.2,kV=1_2_0.2', 'k2vkv', suf='e'))
# param_expr_maps.update(create_job('k2V=-2_6_0.2,kV=2_3_0.2', 'k2vkv', suf='f'))
# pprint(param_expr_maps)

In [88]:
options_map = [{
    'poi'         : 'xsec_br',
    'data'        : 'combData',
    'unblind'     : '',
    'fix'         : "^THEO_XS_fixmu_*=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0,alpha_THEO_XS_PDFalphas_ggFSMHH*=0,alpha_THEO_XS_SCALEMTop_ggFSMHH*=0,THEO_XS_COMBINED_HH_ggF*=0,THEO_XS_PDFalphas_HH_VBF*=0,THEO_XS_PDFalphas_HH_ggF*=0,THEO_XS_SCALE_HH_VBF*=0,alpha_THEO_XS_PDFalphas_ggf=0,alpha_THEO_XS_PDFalphas_vbf=0,alpha_THEO_XS_SCALEMTop_ggf=0,alpha_THEO_XS_SCALEMTop_vbf=0^",
}, {
    'poi'         : 'mu_HH_VBF',
    'data'        : 'combData',
    'unblind'     : '',
    'fix'         : "^THEO_XS_fixmu_*ggFHH=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0,THEO_XS_PDFalphas_HH_VBF*=0,THEO_XS_SCALE_HH_VBF*=0,alpha_THEO_XS_PDFalphas_vbf=0,alpha_THEO_XS_SCALEMTop_vbf=0^",
}
]
command_str_map = {}

condor = False
if condor:
    print('Save job2.txt')
    original_stdout = sys.stdout
    sys.stdout = open('job2.txt', 'w')

for channel in rescaled_ws_paths:
    print(f"# Channel: {channel}")
    input_path  = rescaled_ws_paths[channel]
    for key, expr in param_expr_maps.items():
        if key.split('_')[0] == 'k2v':
            options = options_map[1]
        else:
            options = options_map[0]
        
        if condor and 'final' in key:
            continue
        if not condor and 'job' in key:
            continue
        if 'final' in key:
            outdir = os.path.join(paths['output'], 'xsection_scan', channel, key.replace('_final', ''))
        else:
            outdir = os.path.join(paths['output'], 'xsection_scan', channel, (key+'1')[:key.find('_job')])

        channel_options = {"input_path": input_path, "outdir": outdir, "param_expr": expr, **options}
        channel_command_str = "quickstats limit_scan " + \
                              " ".join([f"--{key} {value}" for key, value in channel_options.items()])
        command_str_map[channel] = channel_command_str


        if not condor:
            print(channel_command_str.replace('^', '"'))
        else:
            print("Arguments =", channel_command_str.replace(' ', '____'))
            print("Queue 1")
    print()

if condor:
    sys.stdout = original_stdout

# Channel: bbbb
quickstats limit_scan --input_path /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_xsec/rescaled/nonres/bbbb/0_kl.root --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_xsec/xsection_scan/bbbb/klambda --param_expr "klambda=-10_15_0.2,klambda=4_7_0.1" --poi xsec_br --data combData --unblind  --fix "THEO_XS_fixmu_*=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0,alpha_THEO_XS_PDFalphas_ggFSMHH*=0,alpha_THEO_XS_SCALEMTop_ggFSMHH*=0,THEO_XS_COMBINED_HH_ggF*=0,THEO_XS_PDFalphas_HH_VBF*=0,THEO_XS_PDFalphas_HH_ggF*=0,THEO_XS_SCALE_HH_VBF*=0,alpha_THEO_XS_PDFalphas_ggf=0,alpha_THEO_XS_PDFalphas_vbf=0,alpha_THEO_XS_SCALEMTop_ggf=0,alpha_THEO_XS_SCALEMTop_vbf=0"
quickstats limit_scan --input_path /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220701_noSgHparam_xsec/rescaled/no

In [28]:
from quickstats.components import ExtendedModel
file='/afs/cern.ch/user/z/zhangr/work//HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbbb/0_kl.root'
model = ExtendedModel(file)
df = model.as_dataframe("constraint")


INFO: Opening file "/afs/cern.ch/user/z/zhangr/work//HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbbb/0_kl.root"
INFO: Loaded workspace "combWS"
INFO: Loaded model config "ModelConfig"
INFO: Activated binned likelihood attribute for ggf_16_dEta_1_Xhh_1_model
INFO: Activated binned likelihood attribute for ggf_16_dEta_1_Xhh_2_model
INFO: Activated binned likelihood attribute for ggf_16_dEta_2_Xhh_1_model
INFO: Activated binned likelihood attribute for ggf_16_dEta_2_Xhh_2_model
INFO: Activated binned likelihood attribute for ggf_16_dEta_3_Xhh_1_model
INFO: Activated binned likelihood attribute for ggf_16_dEta_3_Xhh_2_model
INFO: Activated binned likelihood attribute for ggf_17_dEta_1_Xhh_1_model
INFO: Activated binned likelihood attribute for ggf_17_dEta_1_Xhh_2_model
INFO: Activated binned likelihood attribute for ggf_17_dEta_2_Xhh_1_model
INFO: Activated binned likelihood attribute for ggf_17_dEta_2_Xhh_2_m

KeyError: 'nuisance parameter'